# 要約 
このJupyter Notebookは、Kaggleの「LLM 20 Questions」競技に関連するデータを収集し、分析することを目的としています。具体的には、過去のゲームデータを取得し、特定のゲームエピソードに関連する情報を構造化された形で保存することに取り組んでいます。

### 問題：
ノートブックは、Kaggleのメタデータセットを使用して、特定の競技（episodeId）に関連するエピソードやゲームの情報を収集し、フィルタリングするという問題に取り組んでいます。ゲームの種類に応じて、適切なエピソードを抽出し、正確な結果を得るための処理を実施しています。

### 手法とライブラリ：
1. **ライブラリ**: 
   - `requests`: APIからデータを取得するために使用。
   - `json`: 取得したデータがJSON形式であるため、データの解析に利用。
   - `pandas`: データフレームの作成・操作を行うために使用。
   - `pathlib`: パス操作のために活用。

2. **データ取得と処理手法**:
   - Kaggleのメタデータセットから過去のゲームエピソードを取得するために、APIリクエストを使用。
   - ノートブックでは、CSVファイルを用いて以前のゲームデータを読み込み、新しいゲーム情報を取得し、データフレームに統合。
   - ゲームのフィルタリングには、条件に応じたフィルタリング処理（例えば、検証ゲームを除外）や、データを一時的にメモリに保持するジェネレーターを使用。
   - ゲームの各エピソードを解析し、必要な情報（質問、推測、回答）を抽出してデータフレームに保存。

3. **出力**: 
   - 最終的に、ゲームのデータをCSVファイルに保存し、今後の分析やリーダーボード作成のために使用できる形に整形。

### まとめ:
このノートブックは、Kaggleの「LLM 20 Questions」コンペティションにおけるデータ駆動型の分析を実施するための重要なステップを示しています。自動化されたデータ取得とフィルタリングの手法を用いることで、将来のモデル評価やパフォーマンス向上に貢献することを目指しています。

---


# 用語概説 
以下に、提供されたJupyter Notebookに関連する専門用語の解説を列挙します。初心者がつまずく可能性のある、特に実務経験を持たないと馴染みのない内容に焦点を当てました。

1. **episodeId**:
   - コンペティションにおける特定のプレイセッションを識別するためのユニークな識別子。複数のゲーム（エピソード）が存在するコンペティションでは、各エピソードを区別するために使われる。

2. **submissionId**:
   - 各チームまたはエージェントがコンペティションに提出したモデルや回答の識別子。これを用いて、特定の提出物を追跡したり、評価を行ったりする。

3. **validation game**:
   - 検証用のゲーム。通常、同じエージェントが対戦を行うことで、モデルのパフォーマンスを確認する目的で行われる。

4. **chunk**:
   - 大きなデータファイルやデータセットを小さな部分（チャンク）に分割して処理する手法。これにより、大量のデータを効率的に扱うことができる。

5. **mask**:
   - 特定の条件を満たすデータのインデックスを選択するためのブール型フィルター。データフレーム内の特定の行を抽出する際に用いる。

6. **yield**:
   - Pythonのジェネレーター関数において、関数の実行を一時停止し、特定の値を返すためのキーワード。次回呼び出されると、前回の状態から再開する。

7. **DataFrame**:
   - pandasライブラリで提供される、2次元のデータ構造。表形式のデータを扱うためのもので、行と列から構成される。

8. ***.csvファイル**:
   - Comma-Separated Valuesの略で、データをカンマで区切って保存するためのファイル形式。データのテーブル表現をテキスト形式で扱いたい場合によく使用される。

9. **index_col**:
   - pandasにおけるDataFrameの作成時に、特定の列をインデックスとして指定するための引数。この列は、データの行を識別するために使用される。

10. **concatenate**:
    - 複数のデータ構造を結合する操作。例えば、複数のデータフレームを一つにまとめた新しいデータフレームを作成する場合に用いられる。

11. **observation**:
    - 各プレイの特定の状況や状態を記録するデータ。ゲームのステップや進行状況を示す。

12. **mask.any()**:
    - pandasにおけるメソッドで、ブール配列の中に任意のTrueが存在するかをチェックする。データのフィルタリングに役立つ。

13. **episodeId**:
    - 特定のエピソードを識別するためのID。ゲームの進行状況や結果を確認するために用いられ、各エピソードにはユニークなIDが付与される。

これらの用語は、特にコンペティションやデータ処理に不慣れな初心者にとって、理解が重要なものです。

---


In [ ]:
import requests
import json
import pandas as pd
from pathlib import Path

[Meta kaggleデータセット](https://www.kaggle.com/datasets/kaggle/meta-kaggle)を使用して、コンペティションの`episodeId`を取得します https://www.kaggle.com/competitions/llm-20-questions  
（meta-kaggleが更新されたときに、出所に`[Dataset no longer available]`と表示されることがありますが、それでも機能します）

更新: グエッサー（guesser）とアンサー（answerer）からの`submissionId`を追加しました  
次のステップを参照してください: https://www.kaggle.com/code/waechter/llm-20-questions-leaderbord-analyze-best-agents

⚠️ *最初からフォークして実行しないでください。リクエストが大量になるため、  
代わりにデータセットまたはノートブックの出力を使用してください* ⚠️

# 以前のゲームを読み込む


In [ ]:
df_games = pd.read_csv("/kaggle/input/llm-20-questions-games-dataset/LLM-20Questions-games.csv", index_col='index')
df_games

In [ ]:
def download_game(num, CreateTime, do_print=False):
    url_api = "https://www.kaggleusercontent.com/episodes/{num_episode}.json"
    try:
        r = requests.get(url_api.format(num_episode=num))
        if r.status_code!=200:
            print(f"Error http {r.status_code=} {num=}")
            return {}
        resp = r.json()
        teams = resp["info"]["TeamNames"][0:2], resp["info"]["TeamNames"][2:] # チーム名を取得（guesser_team1, answerer_team1）、(guesser_team2, answerer_team2)
        formated = {}
        for (guesser,answerer), step in zip(teams, resp["steps"][-1][1::2]): # 各チームの最後のステップとして推測者の情報を取得
            keyword, category = step['observation']['keyword'], step['observation']['category']
            if do_print:
                print(f"###\n{guesser=} {answerer=} step {len(step['observation']['answers'])}")
                print(f"{keyword=} {category=}")
                for answer, question, guess in zip(step['observation']['answers'], step['observation']['questions'], step['observation']['guesses']):
                    print(f"\t{question=} {answer=} {guess=}")
            game_index = f"{num}__{guesser}__{answerer}"
            formated[game_index] = {key : step['observation'][key] for key in ['answers', 'questions', 'guesses', 'keyword', 'category']}
            formated[game_index]["guesser"]=guesser
            formated[game_index]["answerer"]=answerer
            formated[game_index]["nb_round"]=len(step['observation']['answers'])
            formated[game_index]["game_num"]=num
            formated[game_index]["guessed"]=keyword in step['observation']['guesses'] # 正しくない可能性あり、keyword_guessedを参照 https://www.kaggle.com/code/waechter/llm-20-questions-leaderbord-analyze-best-agents?scriptVersionId=181553251&cellId=4
            formated[game_index]["CreateTime"]=CreateTime
        return formated
    except Exception as e:
        print("Error parsing", num)
#         print(e)
        return {}

# このコンペティションの新しいエピソードをメタkaggleデータセットから取得


In [ ]:
%%time
meta_path = Path("/kaggle/input/meta-kaggle")
episodes_df = pd.read_csv(meta_path/"Episodes.csv", index_col="Id")

In [ ]:
episodes_df = episodes_df.loc[episodes_df.CompetitionId==61247]
episodes_df

* タイプ4は検証ゲーム（1チーム、同じエージェント両側）
* タイプ1は2チーム、4エージェント、これらのみを保持します


In [ ]:
# 本物のゲームのみを保持する（検証ゲームを除外する）
episodes_df = episodes_df.loc[episodes_df.Type==1,"CreateTime"]

## 大きなファイルを読み込んでフィルタリングするためのジェネレーターを使用
`EpisodeAgents.csv`は一度に読み込むには大きすぎます


In [ ]:
def filter_gen(filepath=meta_path/"EpisodeAgents.csv", chunksize = 10 ** 6):
    episodes_ids = episodes_df.index
    with pd.read_csv(filepath, chunksize=chunksize, index_col="Id") as reader:
        for chunk in reader:
            mask = chunk.EpisodeId.isin(episodes_ids)
            if mask.any():
                yield chunk.loc[mask]

In [ ]:
%%time
agents_df = pd.concat(filter_gen())
agents_df

# 新しいゲームを追加する


In [ ]:
%%time
print(f"以前のゲーム数 {len(df_games)=}")
df_games = pd.concat([df_games]+[pd.DataFrame(download_game(episode_id, date)).T for episode_id, date in episodes_df.items() if episode_id not in df_games.game_num.values])
print(f"更新後 {len(df_games)=}")
df_games

In [ ]:
new_category_games = df_games.loc[(df_games.category =='place') | (df_games.category =='things')]
new_category_games

In [ ]:
keywords_category = pd.DataFrame({keyword.lower():{"keyword":keyword,"category":category} for keyword,category in new_category_games[["keyword","category"]].value_counts().index}).T
keywords_category = keywords_category.sort_index()
keywords_category.to_csv("keywords.csv")

## SubmissionIdを追加


In [ ]:
%%time
for name, group in df_games.groupby(by="game_num"):
    if "guesser_SubmissionId" and "answerer_SubmissionId" in group.columns and (group.guesser_SubmissionId>0).all() and (group.answerer_SubmissionId>0).all():
        continue # すでに完了しています
    
    agents_sub=agents_df[agents_df.EpisodeId==name].sort_values("Index")
    
    if len(agents_sub)!=4:
        print(f"{name=} 利用可能な全ての提出がありません。スキップします。")
        continue
        
    # EpisodeAgentsは各EpisodeIdに対して4行を持ち、インデックスは0-4。出力ログjson [guesser_team1,answerer_team1,guesser_team2,answerer_team2] と同じ順序
    for i, (index, row) in enumerate(group.iterrows()):
        # 2行（各チームに対して1つずつ）
        df_games.loc[index, ["guesser_SubmissionId","answerer_SubmissionId"]] = [agents_sub.loc[agents_sub.Index==i*2,"SubmissionId"].iloc[0], agents_sub.loc[agents_sub.Index==1+i*2,"SubmissionId"].iloc[0]]

df_games[["guesser_SubmissionId","answerer_SubmissionId"]] = df_games[["guesser_SubmissionId","answerer_SubmissionId"]].fillna(0).astype("int")
df_games

### マージが正しく行われたかテスト


In [ ]:
if set(df_games.answerer_SubmissionId) != set(df_games.guesser_SubmissionId):
    print("guesserのみに存在するsub",set(df_games.guesser_SubmissionId) - set(df_games.answerer_SubmissionId))
    print("answererのみに存在するsub",set(df_games.answerer_SubmissionId) - set(df_games.guesser_SubmissionId))
if set(df_games.answerer) != set(df_games.guesser):
    print("guesserのみに存在するチーム",set(df_games.guesser) - set(df_games.answerer))
    print("answererのみに存在するチーム",set(df_games.answerer) - set(df_games.guesser))

In [ ]:
f"ユニークなチーム数: {len(df_games.answerer.unique())} 提出数:{len(df_games.answerer_SubmissionId.unique())}"

In [ ]:
# df_games.answerer_SubmissionId.unique()の各submissionIdについて:
#     guesser = df_games.loc[df_games.guesser_SubmissionId==submissionId,"guesser"].unique()
#     answerer = df_games.loc[df_games.answerer_SubmissionId==submissionId,"answerer"].unique()
#     if len(answerer)>1  or len(guesser)>1:
#         print(f"{submissionId=} で複数のチーム名が見つかりました {guesser=}, {answerer=}")

`submissionId=0`は見つからなかったもので、次のデータ更新時に再度埋める予定です  
残りは名前を更新したチームだと思います


In [ ]:
df_games[["guesser_SubmissionId","guesser"]].value_counts()

In [ ]:
for team in df_games.answerer.unique():
    submissions_set = set(df_games.loc[df_games.guesser==team, "guesser_SubmissionId"]) & set(df_games.loc[df_games.answerer==team, "answerer_SubmissionId"])
    print(f"{team=}, {len(submissions_set)} 提出数")

# 保存


In [ ]:
df_games.to_csv("LLM-20Questions-games.csv", index_label="index")
agents_df.to_csv("EpisodeAgents.csv", index_label="Id")

https://www.kaggle.com/code/waechter/llm-20-questions-leaderbord-analyze-best-agentsで使用


---

# コメント

> ## OminousDude
> 
> とてもとても役立つ、素晴らしい仕事に感謝します！私のモデルのパフォーマンスを確認する手助けをしてくれます。
> 

> 
> 

---

> ## Mario Caesar
> 
> LLMデータセットに関する優れた作業をしてくれて、私はあなたの作業から多くの新しいことを学びました[@waechter](https://www.kaggle.com/waechter)。共有してくれてありがとう😎👊！支持します！！これからも頑張ってください。
> 
> 
> 

---

> ## Zeeshan Latif
> 
> [@waechter](https://www.kaggle.com/waechter) 素晴らしい仕事です！
> 
> 

---

> ## JuHyeon_
> 
> 素晴らしい！共有してくれてありがとう[@waechter](https://www.kaggle.com/waechter) 
> 
> LLM初心者にとって素晴らしい教材です:)
> 
> 高評価😀時間があれば私のノートブックも見て欲しいです！
> 

---

> ## Sara Taghizadeh Milani
> 
> 明らかに多くの思考と努力が込められています。徹底性と明確性から高評価—あなたの今後の作品が楽しみです！
> 
> [@waechter](https://www.kaggle.com/waechter) 
> 

---

> ## Marília Prata
> 
> あなたのデータセットLLM-20Questions-gamesに関する素晴らしい仕事、Waechter。
> 
> 

---